In [1]:
from tabpy.tabpy_tools.client import Client
client = Client('http://localhost:9004/')

In [105]:
#Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [106]:
#Load dataset
url = "https://raw.githubusercontent.com/callxpert/datasets/master/Loan-applicant-details.csv"
names = ['Loan_ID','Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area','Loan_Status']
dataset = pd.read_csv(url, names=names)

In [107]:
dataset

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,Y
4,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267,360,1,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,LP002978,Female,No,0,Graduate,No,2900,0.0,71,360,1,Rural,Y
476,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40,180,1,Rural,Y
477,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253,360,1,Urban,Y
478,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187,360,1,Urban,Y


In [108]:
dataset.to_excel("credit_scoring.xlsx")

In [7]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']
le = LabelEncoder()
for i in var_mod:
    dataset[i] = le.fit_transform(dataset[i])

In [22]:
array = dataset.values
X = array[:,6:11]
Y = array[:,12]
Y=Y.astype('int')
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.2, random_state=7)

In [24]:
model = LogisticRegression()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print(accuracy_score(y_test, predictions))

0.7708333333333334


In [109]:
def predict_loan(applicant_income, co_applicant_income, loan_amount, loan_amount_term, credit_history):
    X = np.column_stack([
        applicant_income, co_applicant_income, loan_amount, loan_amount_term, credit_history
    ])
    pred = model.predict(X).tolist()[0]
    if pred == 1:
        return 'Approved'
    elif pred == 0:
        return 'Rejected'

In [110]:
predict_loan(10,50,10,30,1)

'Approved'

In [112]:
client.remove('PredictLoan')

In [114]:
client.deploy('PredictLoanv1', predict_loan, 'predict loan is approved or not based on 5 variables')